In [ ]:
library(tidyverse) # metapackage of all tidyverse packages
library(plyr)
library(modeest)
library(ggplot2)
library(ggpubr)
library(cowplot)
library(randomForest)

In [ ]:
path = "../input/hepatitis/hepatitis.csv"
columns = c("class", "age", "sex", "steroid", "antivirals", "fatigue", "malaise", "anorexia", "liver_big", "liver_firm", "spleen_palpable", "spiders", "ascites", "varices", "bilirubin", "alk_phosphate", "sgot", "albumin", "protime", "histology")
data = read.csv(path, header = F, sep=",", col.names = columns)

In [ ]:
head(data)

In [ ]:
dataHip <- data

In [ ]:
parsingNA <- function(col){
     if(is.factor(col)){
        col <- revalue(col, c("?" = NA))
     }
    if(nlevels(col) > 2){
        col <- as.numeric(levels(col))[col]
    }
    return (col)
}

In [ ]:
# Recorremos cada columna para reemplazar los símbolos ? y dejarlos como NA
# Esto con el objeto de que Mice pueda reeconocer los datos perdidos y aplicar los métodos correspondientes.
for(i in names(dataHip)){
  dataHip[[i]] <- parsingNA(dataHip[[i]])
}

In [ ]:
#Chequeamos la cantidad de datos perdidos
colSums(is.na(dataHip))

In [ ]:
# Aplicamos la imputación
# Poster
library(mice)
init = mice(dataHip, maxit=0) 
meth = init$method
# Descartamos estos atributos para la imputación
meth[c("age", "class", "sex", "antivirals", "histology")]=""
# Se aplica método de regresión logística para atributos con 2 niveles
meth[c("steroid", "fatigue", "malaise", "anorexia", "liver_big", "liver_firm", "spleen_palpable", "spiders", "ascites", "varices")]="logreg"
# Se aplica método de regresión logística politómica
meth[c("bilirubin", "alk_phosphate", "sgot", "albumin", "protime")]="norm"
predM = init$predictorMatrix
set.seed(103)
imputed = mice(dataHip, method=meth, predictorMatrix=predM, m=5)

In [ ]:
dataImputed <- complete(imputed)
dataImputedNonConverted <- dataImputed

In [ ]:
str(dataImputed)
datosDiscretos <- dataImputed

In [ ]:
convertToInt <- function(col){
    col <- as.integer(levels(col))[col]
    return (col)
}

In [ ]:
convertToNumeric <- function(col){
    col <- as.numeric(levels(col))[col]
    return (col)
}

In [ ]:

# Convertimos los atributos que son categóricos a enteros
for(i in names(dataImputed)){
  if(is.factor(dataImputed[[i]])){
      dataImputed[[i]] <- convertToNumeric(dataImputed[[i]])
  }
}

In [ ]:
colSums(is.na(dataImputed))

In [ ]:
datos <- dataImputed

### transformas la class en tipo factor

In [ ]:
datos$class = factor(datos$class, levels = c(1,2), labels = c("Fallecido", "Vivo"))

In [ ]:
summary(datos)

In [ ]:
str(datos)

In [ ]:
library("C50")
library("caret")

## Generamos conjunto de entrenamiento y pruebas

In [ ]:
# arreglo con diferentes porcentajes de entrenamiento
p_train_arr = list(0.7, 0.75, 0.80, 0.85, 0.9)

In [ ]:
plotModel <- function(data, p_train, seed, trials = 1){
    set.seed(seed)

    training.index = createDataPartition(data$class, p=p_train)$Resample1

    training.set = data[training.index, ]

    test.set = data[-training.index, ]
    
    tree = C5.0(class ~ ., training.set, trials = trials) #Probar , rules = T
    #Modelo basado en reglas
    tree.rules = C5.0(x = training.set[, -1], y = training.set$class, rules = TRUE)

    #Predicción de clases con instancia de pruebas
    tree.pred.class = predict(tree, test.set[,-1], type = "class")

    #Probabilidad por clases
    tree.pred.prob = predict(tree, test.set[,-1], type = "prob")
    
    print("--------------------DECISITION TREE--------------------")
    print(paste("Porcentaje de entrenamiento: ", p_train))
    print(summary(tree))
    plot(tree)
    print("--------------------REGLAS--------------------")
    print(summary(tree.rules))
    print("--------------------CM--------------------")
    conf.matrix.tree = confusionMatrix(table(test.set$class, tree.pred.class))
    print(conf.matrix.tree)
    print("------------------- RANDOM FOREST----------------------")
    rf = randomForest(class ~ ., data=training.set, ntree = 500, importance=TRUE, proximity=TRUE, ntry=10)
    rf.pred.class = predict(rf, test.set[,-1], type = "class")
    conf.matrix.rf = confusionMatrix(table(test.set$class, rf.pred.class))
    print(conf.matrix.rf)
    varImpPlot(rf)
}

### Generamos múltiples instancias de entrenamiento

In [ ]:
#Hubiese sido elegante con un for, pero los malditos plot sin async
plotModel(datos, as.numeric(p_train_arr[1]), 1234, 1)

In [ ]:
plotModel(datos, as.numeric(p_train_arr[2]), 1234)

In [ ]:
plotModel(datos, as.numeric(p_train_arr[3]), 1234)

In [ ]:
plotModel(datos, as.numeric(p_train_arr[4]), 1234)

In [ ]:
plotModel(datos, as.numeric(p_train_arr[5]), 1234)

### Al parecer los mejores grupos de entrenamiento, está entorno al 80% del uso de los datos como conjunto de entrenamiento

In [ ]:
plotModel(datos, 0.75, 1234, 5)

### Al parecer aunque se mejore el accuracy con trial diferentes de 1, el árbol contiene clasificadores que no son acordes a la realidad. Ya que un paciente que no presenta acumulación de líquido seroso en la zona abdominal y que es mayor a 28 años por ejemplo, no debería ser consecuente a la muerte. Por lo tanto se sugiere mejorar este modelo.

Con el 50% de los datos, también se obtiene un accuracy importante

In [ ]:
plotModel(datos, 0.5, 1234, 5)

> **This is a fundamental outcome of the random forest and it shows, for each variable, how important it is in classifying the data. The Mean Decrease Accuracy plot expresses how much accuracy the model losses by excluding each variable. The more the accuracy suffers, the more important the variable is for the successful classification. The variables are presented from descending importance. The mean decrease in Gini coefficient is a measure of how each variable contributes to the homogeneity of the nodes and leaves in the resulting random forest. The higher the value of mean decrease accuracy or mean decrease Gini score, the higher the importance of the variable in the model.**

In [ ]:
head(datos)

In [ ]:
datosFiltered <- datos[, c(1,2, 9,12,15, 16,17,18,19,20)]

In [ ]:
head(datosFiltered)

In [ ]:
plotModel(datosFiltered, 0.5, 1234, 5)

## Si aplicamos balance por Synthetic Data Generation y filtramos columnas de mayor importancia, según literatura y experimentación anterior

In [ ]:
library(ROSE)

In [ ]:
datos.syn <- ROSE(class ~ ., data = datos, seed = 1)$data
table(datos.syn$class)

In [ ]:
datos.syn.filtered <- datos.syn[, c(1,2, 9,12,15, 16,17,18,19,20)]

In [ ]:
plotModel(datos.syn.filtered, 0.8, 1234, 5)

### Notamos que tenemos un acc ~83% y una sens ~91% que es una mejora bastante significativa, y por lo demas se tiene un árbol algo más realista del punto de vista de la literatura

In [ ]:
plotModel(datos.syn.filtered, 0.5, 1234, 5)

Notar además que en todos los RF anteriores, para el conjunto de datos no filtrados, los atributos más relevantes, son similares para diferentes conjuntos de entrenamientos, como por ejemplo: ascites, protime, albumin, bilirubin, alk_phospate, varices, histology, sgot.

In [ ]:
plotModel(datos.syn.filtered, 0.7, 1234, 5)

Nota2: las variables con menos importancia detectadas con estos métodos, fueron por ejemplo el sexo, anorexia, antivirals, liver_big, steroid, fatigue

In [ ]:
plotModel(datos.syn.filtered, 0.8, 1234, 5)

In [ ]:
plotModel(datos.syn.filtered, 0.8, 1234, 10)

In [ ]:
plotModel(datos.syn.filtered, 0.7, 1234, 10
         )